# 🎌 Anime 4K Upscale - ONE CLICK

**Chỉ cần:**
1. Sửa `INPUT_DIR` và `OUTPUT_DIR` ở Cell 1
2. Nhấn **Runtime → Run all** (`Ctrl+F9`)
3. Xong! ☕

In [ ]:
#@title ⚙️ CẤU HÌNH (sửa ở đây)
INPUT_DIR = "/content/drive/MyDrive/Movies/Input" #@param {type:"string"}
OUTPUT_DIR = "/content/drive/MyDrive/Movies/Output" #@param {type:"string"}
MODE = "anime_4k" #@param ["anime_4k", "anime_fast", "ultra_fast"]

#@markdown ---
#@markdown ### Multi-Colab (optional)
WORKER_ID = 1 #@param {type:"integer"}
TOTAL_WORKERS = 1 #@param {type:"integer"}

In [ ]:
#@title 🚀 CHẠY TẤT CẢ (tự động)
import os, glob, subprocess, json, time, pickle
from datetime import timedelta

# ====== MOUNT DRIVE ======
print("📂 Mounting Drive...")
from google.colab import drive
drive.mount('/content/drive')

# ====== CHECK GPU ======
print("\n🖥️ GPU:")
!nvidia-smi --query-gpu=name,memory.free --format=csv

# ====== INSTALL ======
print("\n📦 Installing...")
if not os.path.exists('/content/Real-ESRGAN/weights/realesr-animevideov3.pth'):
    !git clone https://github.com/xinntao/Real-ESRGAN.git 2>/dev/null
    %cd /content/Real-ESRGAN
    !pip install basicsr facexlib gfpgan -q
    !pip install -r requirements.txt -q
    !python setup.py develop 2>&1 | tail -1
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-animevideov3.pth -P weights/ -q
print("✅ Tools ready!")

# ====== CONFIG ======
MODES = {
    'anime_4k': {'tile': 192, 'tile_pad': 24, 'crf': 22, 'preset': 'fast'},
    'anime_fast': {'tile': 128, 'tile_pad': 16, 'crf': 24, 'preset': 'veryfast'},
    'ultra_fast': {'tile': 64, 'tile_pad': 8, 'crf': 28, 'preset': 'ultrafast'}
}
cfg = MODES[MODE]
PROGRESS = f"/content/drive/MyDrive/Movies/.progress_{WORKER_ID}.pkl"

# ====== SCAN ======
print(f"\n🔍 Scanning {INPUT_DIR}...")

def get_info(p):
    try:
        r = subprocess.run(['ffprobe','-v','quiet','-print_format','json','-show_format','-show_streams',p], capture_output=True, text=True, timeout=30)
        d = json.loads(r.stdout)
        v = next((s for s in d.get('streams',[]) if s.get('codec_type')=='video'), {})
        return {'w': int(v.get('width',0)), 'h': int(v.get('height',0)), 'dur': float(d.get('format',{}).get('duration',0))}
    except: return {'w':0,'h':0,'dur':0}

# Load progress
done = set()
if os.path.exists(PROGRESS):
    with open(PROGRESS, 'rb') as f: done = pickle.load(f)
    print(f"📂 Resume: {len(done)} videos đã xong")

# Find videos
all_vids = []
for ext in ('*.mp4','*.mkv','*.avi','*.mov','*.webm'):
    all_vids.extend(glob.glob(os.path.join(INPUT_DIR,'**',ext), recursive=True))
all_vids = sorted(all_vids)

# Worker split
my_vids = [v for i,v in enumerate(all_vids) if (i % TOTAL_WORKERS) == (WORKER_ID - 1)]

# Filter
to_process = []
for p in my_vids:
    name = os.path.basename(p)
    out = os.path.join(OUTPUT_DIR, name.rsplit('.',1)[0] + '_4K.mkv')
    info = get_info(p)
    if name in done or (os.path.exists(out) and os.path.getsize(out) > 1024*1024) or info['w'] >= 3800:
        print(f"⏭️ Skip: {name}")
    else:
        to_process.append({'path': p, 'name': name, 'info': info})
        print(f"✅ Queue: {name} ({info['w']}x{info['h']})")

print(f"\n📊 {len(to_process)} videos to process")

# ====== PROCESS ======
if not to_process:
    print("\n✅ Không có video nào cần xử lý!")
else:
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    os.makedirs('/content/temp', exist_ok=True)
    
    print(f"\n{'='*50}")
    print(f"🚀 BẮT ĐẦU - Mode: {MODE}")
    print(f"{'='*50}")
    
    for idx, item in enumerate(to_process, 1):
        p, name, info = item['path'], item['name'], item['info']
        out = os.path.join(OUTPUT_DIR, name.rsplit('.',1)[0] + '_4K.mkv')
        temp = f'/content/temp/{idx}.mp4'
        
        print(f"\n🎌 [{idx}/{len(to_process)}] {name}")
        start = time.time()
        
        try:
            # Upscale
            %cd /content/Real-ESRGAN
            !python inference_realesrgan_video.py \
                -n realesr-animevideov3 \
                -i "{p}" -o "{temp}" -s 2 \
                --tile {cfg['tile']} --tile_pad {cfg['tile_pad']} \
                --suffix "" --fp32
            
            if not os.path.exists(temp): raise Exception("Upscale failed")
            
            # Encode
            !ffmpeg -y -i "{p}" -i "{temp}" \
                -map 1:v -map 0:a? -map 0:s? \
                -c:v libx265 -crf {cfg['crf']} -preset {cfg['preset']} -tag:v hvc1 \
                -c:a copy -c:s copy \
                -map_metadata 0 -map_chapters 0 \
                "{out}" -loglevel warning -stats
            
            # Save progress
            done.add(name)
            with open(PROGRESS, 'wb') as f: pickle.dump(done, f)
            
            elapsed = time.time() - start
            size = os.path.getsize(out) / (1024**3)
            print(f"✅ Done: {timedelta(seconds=int(elapsed))} | {size:.2f}GB")
            
        except Exception as e:
            print(f"❌ Error: {e}")
        finally:
            if os.path.exists(temp): os.remove(temp)
    
    print(f"\n{'='*50}")
    print(f"🏁 HOÀN THÀNH!")
    print(f"📂 Output: {OUTPUT_DIR}")
    print(f"{'='*50}")